In [10]:
import numpy as np
import jax
import jax.numpy as jnp
import scipy.signal as ss

jax.config.update("jax_enable_x64", True)

import discovery.flow as dsf
from flowjax.flows import triangular_spline_flow
from flowjax.distributions import StandardNormal
from ripplegw.waveforms.IMRPhenomD import gen_IMRPhenomD_hphc

print("JAX devices:", jax.devices())

JAX devices: [CudaDevice(id=0)]


## Set up LIGO Data

In [11]:
# ==========================================
# STEP 1: Load data with windowing + PSD
# ==========================================
from scipy.signal.windows import tukey

data_raw = np.loadtxt("gw250114_20s.txt")
noise_raw = np.loadtxt("gw250114_50s_offsource.txt")

t, h = data_raw[:, 0], data_raw[:, 1]
t_noise, h_noise = noise_raw[:, 0], noise_raw[:, 1]
dt = float(t[1] - t[0])
fs = 1.0 / dt
fs_noise = 1/(float(t_noise[1] - t_noise[0]))

# Apply Tukey window to suppress spectral leakage from low-freq seismic noise
window = tukey(len(h), alpha=0.1)
h_win = h * window

f = jnp.fft.rfftfreq(len(h), d=dt)
d_f = jnp.fft.rfft(h_win) * dt   # windowed FFT
df = float(f[1] - f[0])

# PSD from off-source noise (welch handles its own windowing internally)
f_psd, psd_val = ss.welch(h_noise, fs=fs_noise, nperseg=int(fs_noise * 2))
psd = jnp.interp(f, f_psd, psd_val)

flow_freq, fhigh = 20.0, 512.0
mask = (f >= flow_freq) & (f <= fhigh)

# Sanity check: noise-only inner product should be ~2*T*BW = 2*20*492 ≈ 19680
d_d = 4.0 * jnp.real(jnp.sum(jnp.abs(d_f[mask])**2 / psd[mask]) * df)
print(f"dt={dt}, fs={fs}, df={df}")
print(f"Freq points: {len(f)}, in band: {mask.sum()}")
print(f"<d|d> = {d_d:.1f}  (expected ~19680 for noise-only)")
print(f"noise-only log-lik = {-0.5*d_d:.1f}")

dt=0.000244140625, fs=4096.0, df=0.05
Freq points: 40961, in band: 9841
<d|d> = 21160.8  (expected ~19680 for noise-only)
noise-only log-lik = -10580.4


## Sanity Check
Use ripplegw for the waveform generation because it returns an output differentiable in jax

In [12]:
# ==========================================
# STEP 2: Test waveform + likelihood with known-good params
# ==========================================
# ripplegw params: [Mc, eta, chi1, chi2, dist_mpc, tc, phic, iota]
test_params = jnp.array([30.0, 0.25, 0.0, 0.0, 400.0, 0.0, 0.0, 0.0])

f_nz = f[1:]
hp, hc = gen_IMRPhenomD_hphc(f_nz, test_params, flow_freq)

h_det = jnp.concatenate([jnp.array([0j]), hp])
h_det = jnp.where(f < flow_freq, 0j, h_det)
res = d_f[mask] - h_det[mask]
ll = -0.5 * 4.0 * jnp.real(jnp.sum(jnp.conj(res) * res / psd[mask]) * df)

print(f"hp max abs: {jnp.max(jnp.abs(hp)):.2e}")
print(f"Log-likelihood at test point: {ll:.2f}")
print(f"Log-lik ratio vs noise-only: {ll - (-0.5*d_d):.2f}")

hp max abs: 1.09e-19
Log-likelihood at test point: -13788.61
Log-lik ratio vs noise-only: -3208.22


## Define LIGO Model that can be used as direct input into flow
Since the discovery package is created for Pulsar Timing Array data, the Pulsar data classes are not applicable to LIGO data which contains strain, rather than correlations. However, we can still use the `discovery.flow` because it has been specified from the `Flowjax` built to do many things, among them variational inference.

Variational inference allows us to take LIGO strain data and a physical waveform model as input, use a normalizing flow model to learn the shape of the posterior distribution that best explains the data, and optimize this approximation through gradient-based training.

`discovery.flow` module has an elboLoss function which takes as input a target (parameters and ranges) and the number of samples (num params) which is used for the loss calculation. This loss is then fed as an input to the `dsf.variationalFit` trainer which the model uses to optimize the posterior distribution fit. Below, `LIGOModel` models the waveform, and `FlowTarget` is an object describing the parameters we're optimizing for.

Regarding `_make_safe_target()` - I was running into errors because the loss was continuously returning nan values and the model was then not returning any physical values. From what I could see, the loss would be nan when the model returned unphysical values from the parameter space but then this nan would just be passed to each iteration so it never worked. Simply using set_nan_to_num (setting all nans = 0) also didn't work (according to claude, this is because of how the flow code works, if one param is nan they all become nan forever. I haven't dug deep into this, but see the explanation below). Adding in a specific function that ensures that nans for certain parameters don't poison the entire workflow seems to fix this. We get what look like physical parameter estimates. Time to optimize.

In [13]:
# ==========================================
# STEP 3: Define model and target, test values + gradients
# ==========================================
class LIGOModel:
    def __init__(self):
        self.f, self.d_f, self.df = f, d_f, df
        self.psd, self.mask = psd, mask
        self.f_ref, self.f_min = flow_freq, flow_freq

    def get_waveform(self, params):
        f_nz = self.f[1:]
        hp, hc = gen_IMRPhenomD_hphc(f_nz, params, self.f_ref)
        h_det = jnp.concatenate([jnp.array([0j]), hp])
        h_det = jnp.where(self.f < self.f_min, 0j, h_det)
        # Replace NaN with 0 so bad parameter combos yield noise-only
        # likelihood (finite value + finite gradient) instead of poisoning
        # the entire batch gradient average
        h_det = jnp.nan_to_num(h_det, nan=0.0)
        return h_det

    def log_likelihood(self, params):
        h = self.get_waveform(params)
        res = self.d_f[self.mask] - h[self.mask]
        snr_sq = 4.0 * jnp.real(jnp.sum(jnp.conj(res) * res / self.psd[self.mask]) * self.df)
        return -0.5 * snr_sq


def _make_safe_target(log_lik_fn, map_fn, prior_fn):
    """Wrap a target function with NaN-safe gradients via custom_vjp.

    The problem: ripplegw can produce NaN *internally* for certain parameter
    combos. nan_to_num on the output fixes the forward value, but the backward
    pass still propagates NaN gradients through ripplegw's internals.  Since
    value_and_grad_ElboLoss averages gradients over all samples, a single NaN
    gradient poisons the entire batch -> every subsequent step is NaN.

    The fix: custom_vjp lets us intercept the backward pass and replace any
    NaN gradient entries with 0.  A zero gradient means 'this sample contributes
    nothing to the parameter update', which is the correct behaviour for a
    sample that landed in an unphysical region.
    """
    def _raw(z):
        phys = map_fn(z)
        return log_lik_fn(phys) + prior_fn(z)

    @jax.custom_vjp
    def _safe(z):
        return jnp.nan_to_num(_raw(z), nan=-1e10, posinf=-1e10, neginf=-1e10)

    def _fwd(z):
        val = _safe(z)
        return val, z

    def _bwd(z, g):
        grad = jax.grad(_raw)(z)
        grad = jnp.nan_to_num(grad, nan=0.0, posinf=0.0, neginf=0.0)
        return (grad * g,)

    _safe.defvjp(_fwd, _bwd)
    return _safe


class FlowTarget:
    """Target distribution for the normalizing flow.

    Maps unconstrained z in R^8 to the 8 ripplegw IMRPhenomD parameters:
        [Mc, eta, chi1, chi2, dist, tc, phic, iota]
    via a tanh transform with specified bounds.
    """

    param_names = ['Mc', 'eta', 'chi1', 'chi2', 'dist', 'tc', 'phic', 'iota']

    def __init__(self, model):
        self.model = model
        self.dim = 8
        # Bounds in ripplegw order: [Mc, eta, chi1, chi2, dist, tc, phic, iota]
        # Tightened around GW250114 expected values for stable training
        self.a = jnp.array([25.0,  0.15, -0.5, -0.5, 100.0, -0.2,   0.0,  0.0])
        self.b = jnp.array([40.0,  0.25,  0.5,  0.5, 1000.0, 0.2, 2*jnp.pi, jnp.pi])
        # Build NaN-safe callable (custom_vjp sanitises both value and gradient)
        self._safe_fn = _make_safe_target(model.log_likelihood, self.map_to_physical, self.logprior)

    def map_to_physical(self, z):
        """Map unconstrained z to bounded physical parameters via tanh."""
        return 0.5 * (self.b + self.a + (self.b - self.a) * jnp.tanh(z))

    def samples_to_physical(self, z_batch):
        """Batch version of map_to_physical for post-training analysis."""
        return jax.vmap(self.map_to_physical)(z_batch)

    def logprior(self, z):
        """Log-Jacobian of the tanh transform (uniform prior in physical space)."""
        return jnp.sum(jnp.log(2.0) - 2.0 * jnp.logaddexp(z, -z))

    def __call__(self, z):
        return self._safe_fn(z)


model = LIGOModel()
target = FlowTarget(model)

# Test values and gradients
key = jax.random.key(0)
z_test = jax.random.normal(key, (5, target.dim))
grad_fn = jax.grad(target)

# sanity check
print("Target values and gradients on random z samples:")
for i in range(5):
    z = z_test[i]
    phys = target.map_to_physical(z)
    val = target(z)
    g = grad_fn(z)
    names = target.param_names
    param_str = ", ".join(f"{names[j]}={phys[j]:.3f}" for j in range(8))
    print(f"  {param_str}")
    print(f"    -> target={val:.2e}, max|grad|={jnp.max(jnp.abs(g)):.2e}, any_nan={bool(jnp.any(jnp.isnan(g)))}")

Target values and gradients on random z samples:
  Mc=30.978, eta=0.167, chi1=0.474, chi2=0.093, dist=586.311, tc=-0.071, phic=5.751, iota=2.083
    -> target=-1.11e+04, max|grad|=1.04e+03, any_nan=False
  Mc=33.135, eta=0.165, chi1=0.392, chi2=-0.458, dist=708.327, tc=0.197, phic=3.936, iota=1.137
    -> target=-1.09e+04, max|grad|=5.82e+02, any_nan=False
  Mc=28.837, eta=0.218, chi1=-0.097, chi2=0.453, dist=576.541, tc=0.028, phic=3.727, iota=1.644
    -> target=-1.09e+04, max|grad|=5.15e+02, any_nan=False
  Mc=25.838, eta=0.190, chi1=0.403, chi2=-0.489, dist=320.544, tc=0.059, phic=0.570, iota=1.591
    -> target=-1.15e+04, max|grad|=1.82e+03, any_nan=False
  Mc=38.923, eta=0.151, chi1=-0.376, chi2=-0.444, dist=178.886, tc=0.152, phic=1.885, iota=2.148
    -> target=-1.43e+04, max|grad|=6.98e+03, any_nan=False


### Training & Run
The initial loss is calculated using the discovery module ` value_and_grad_ElboLoss` taking the target from above as input. The model is trained using the discovery module `VariationalFit` taking the flow, loss, optimizer and a few other parameters as inputs. Annealing schedule is kept the same as example.

In [14]:
# ==========================================
# STEP 4: Training run
# ==========================================
import optax

rng = jax.random.key(42)
flow_key, train_key = jax.random.split(rng)

flow = triangular_spline_flow(
    flow_key,
    base_dist=StandardNormal((target.dim,)),
    flow_layers=12,
    knots=10
)

# num_samples=16 runs in parallel on GPU, multibatch=4 loops sequentially
# -> effective batch size of 64 but with more stable per-minibatch gradients
loss_fn = dsf.value_and_grad_ElboLoss(target, num_samples=16)


trainer = dsf.VariationalFit(
    flow,
    loss_fn=loss_fn,
    multibatch=4,
    learning_rate=1e-2,
    annealing_schedule=lambda i: min(1.0, 0.01 + 0.99 * i / 500),
    show_progress=True
)

print("Training run (100 steps)...")
train_key, trained_flow = trainer.run(train_key, steps=100)
print(f"Final loss: {trainer.losses[-1]:.2f}")
print(f"Best loss:  {min(trainer.losses):.2f} at iter {trainer.losses.index(min(trainer.losses))}")

Training run (100 steps)...


100%|██████████| 100/100 [00:20<00:00,  4.82it/s, loss=3765.57]

Final loss: 3765.57
Best loss:  307.09 at iter 0


In [15]:
# ==========================================
# STEP 5: Extract results and visualise posteriors
# ==========================================
import pandas as pd
import corner
import matplotlib.pyplot as plt

# Sample from trained flow
sample_key = jax.random.key(123)
z_samples = trained_flow.sample(sample_key, sample_shape=(16384,))
phys_samples = np.array(target.samples_to_physical(z_samples))

# Build DataFrame with ripplegw parameter names
post = pd.DataFrame(phys_samples, columns=target.param_names)

# Derive component masses and mass ratio from Mc and eta
#   M_total = Mc / eta^(3/5),  m2 = (M - sqrt(M^2 - 4*M^2*eta))/2,  m1 = M - m2
Mc_s, eta_s = post['Mc'].values, post['eta'].values
M_total = Mc_s / eta_s**0.6
m2 = 0.5 * (M_total - np.sqrt(np.clip(M_total**2 * (1 - 4*eta_s), 0, None)))
m1 = M_total - m2
post['m1'] = m1
post['m2'] = m2
post['q'] = m2 / m1

# Print summary statistics (median and 90% credible interval)
print("=" * 60)
print("Posterior summary (median [5th, 95th percentile])")
print("=" * 60)
for col in ['Mc', 'eta', 'm1', 'm2', 'q', 'chi1', 'chi2', 'dist', 'tc', 'phic', 'iota']:
    lo, med, hi = np.percentile(post[col], [5, 50, 95])
    print(f"  {col:>6s} = {med:8.3f}  [{lo:8.3f}, {hi:8.3f}]")

print("\nPublished GW250114 values for comparison:")
print("  Mc_det ~ 30.3-31.6 Msun")
print("  m1     ~ 32.6-34.9 Msun")
print("  m2     ~ 30.6-33.2 Msun")
print("  q      > 0.91")

Posterior summary (median [5th, 95th percentile])
      Mc =   40.000  [  25.000,   40.000]
     eta =    0.248  [   0.150,    0.250]
      m1 =   46.756  [  28.717,  101.910]
      m2 =   28.481  [  14.341,   45.948]
       q =    0.853  [   0.225,    1.000]
    chi1 =   -0.500  [  -0.500,    0.500]
    chi2 =   -0.500  [  -0.500,    0.500]
    dist = 1000.000  [ 100.000, 1000.000]
      tc =   -0.200  [  -0.200,    0.200]
    phic =    0.000  [   0.000,    6.283]
    iota =    0.000  [   0.000,    3.142]

Published GW250114 values for comparison:
  Mc_det ~ 30.3-31.6 Msun
  m1     ~ 32.6-34.9 Msun
  m2     ~ 30.6-33.2 Msun
  q      > 0.91


# Appendix : Claude's explanation of the nan problem

The NaN problem explained
The value_and_grad_ElboLoss in flow.py:43 does:

```
vals, grad = eqx.filter_vmap(eqx.filter_value_and_grad(theloss), ...)(...)
return vals.mean(), jax.tree_util.tree_map(lambda array: jnp.mean(array, axis=0), grad)
```

It computes 16 independent (value, gradient) pairs and averages them. If even one sample produces a NaN gradient, jnp.mean over gradients = NaN → the optimizer applies a NaN update → all flow parameters become NaN → every subsequent step is NaN forever.

The old nan_to_num on the forward output didn't help because jnp.nan_to_num has zero gradient at NaN points — it doesn't intercept the backward pass. When ripplegw produces NaN internally during intermediate computations, the gradient through those intermediate values is NaN even if the final output was clamped.

The fix: jax.custom_vjp
The new _make_safe_target wrapper uses jax.custom_vjp to intercept both the forward and backward pass:

Forward (_fwd): computes the target value, replaces NaN with -1e10 (penalty)
Backward (_bwd): computes the gradient normally via jax.grad(_raw), then applies nan_to_num to replace any NaN gradient entries with 0
